## Import libs

In [3]:
import numpy as np
import cv2
import glob
import os
from skimage.feature import hog
from skimage import exposure

## Preprocessing images

In [11]:
def clear_folder():
    roots = ['../data/ratio_h_features', '../data/HOG_features', '../data/all_features']
    total_files = 0

    for root in roots:
        for path in os.listdir(root):
            os.remove(os.path.join(root, path))
            total_files += 1
    
    print("Removed {} files".format(total_files))

In [12]:
rootdir = '../data/Raw/Training/'
total_img = 0
len_HOG_feature = None
len_hist_feature = None

# Clear feature folders
clear_folder()

# Initialize parameters for HOG - feature extraction algorithm

for path in glob.glob(f'{rootdir}/*/**'):
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (96, 128), interpolation = cv2.INTER_AREA)
    gray_arr = np.asarray(gray)

    # Ratio hair feature
    # img_black = np.zeros((128, 96), dtype = np.uint8)
    count = 0
    for i in range(0,128):
        for j in range(0,96):
            if gray_arr[i][j] <= 51 :
                # img_black[i][j] = 255
                count += 1
    ratio_h = [count / (128*96)]

    # HOG features
    hog_features, hog_image = hog(gray_arr, orientations=8, pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2), visualize = True)

    # Concatenate histograms features and HOG features
    merged_features = np.concatenate((hog_features, ratio_h))

    # Save feature
    np.save('../data/ratio_h_features/' + path.split('\\')[-1].split('.')[0], ratio_h)
    np.save('../data/HOG_features/' + path.split('\\')[-1].split('.')[0], hog_features)
    np.save('../data/all_features/' + path.split('\\')[-1].split('.')[0], merged_features)

    # Count number of images
    total_img += 1
    len_HOG_feature = len(hog_features)

### DONE
print("Length of HOG feature: {}".format(len_HOG_feature))
print("Length of hist_all_HOG feature: {}".format(len_HOG_feature + 1))
print("Processed: {} images".format(total_img))

Removed 100 files
Length of HOG feature: 5280
Length of hist_all_HOG feature: 5281
Processed: 100 images


## Calculate L2

In [4]:
def get_features(img, method):

    # Convert images to 
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (96, 128), interpolation = cv2.INTER_AREA)
    gray_arr = np.asarray(gray)

    # Ratio hair feature
    # img_black = np.zeros((128, 96), dtype = np.uint8)
    count = 0
    for i in range(0,128):
        for j in range(0,96):
            if gray_arr[i][j] <= 51 :
                # img_black[i][j] = 255
                count += 1
    ratio_h = [count / (128*96)]

    # HOG features
    hog_features, hog_image = hog(gray_arr, orientations=8, pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2), visualize = True)

    # Concatenate histograms features and HOG features
    merged_features = np.concatenate((hog_features, ratio_h))

    if method == "Ratio hair":
        return ratio_h
    elif method == "HOG":
        return hog_features
    else:
        return merged_features

In [14]:
def prediction(img, method):

    #Intialize params
    ratio_h_path = "../data/ratio_h_features/"
    hog_path = "../data/HOG_features/"
    all_features_path = "../data/all_features/"

    val = []
    idx = []

    current_features = get_features(img, method)
    print(len(current_features))

    if method == "Ratio hair":
        for path in glob.glob(f'{ratio_h_path}/*'):
            ratio_h_tmp = np.load(path)
            dist = np.linalg.norm(ratio_h_tmp - current_features)
            val.append(dist)
            idx.append(path)

    elif method == "HOG":
        for path in glob.glob(f'{ratio_h_path}/*'):
            hog_tmp = np.load(path)
            dist = np.linalg.norm(hog_tmp - current_features)
            val.append(dist)
            idx.append(path)

    else:
        for path in glob.glob(f'{ratio_h_path}/*'):
            all_features_tmp = np.load(path)
            dist = np.linalg.norm(all_features_tmp - current_features)
            val.append(dist)
            idx.append(path)

    nearest_img = np.argsort(val)[:3]
    res_nu = 0
    for img_path in nearest_img:
        if 'nu' in idx[img_path]:
            res_nu += 1
    
    return "Nu" if res_nu > 1 else "Nam"

In [ ]:
test_path = "../data/Raw/Test/"
for img_path in glob.glob(f'{test_path}/*'):
    img = cv2.imread(img_path)
    predicted_label = prediction(img, method="Ratio hair")
    print(img_path, " ---- ", predicted_label)

In [18]:
truth = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
prediction_all_feature = [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
prediction_hog = [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
prediction_ratio = [0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0]

In [22]:
from sklearn.metrics import precision_score, recall_score, f1_score

print(f1_score(truth, prediction_hog, average='binary'))
print(f1_score(truth, prediction_ratio, average='binary'))
print(f1_score(truth, prediction_all_feature, average='binary'))

0.25
0.47619047619047616
0.25
